# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 **이전 처리결과를 입력으로 받아 처리한 후 다음 단계로 결과를 전달**한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행등을 순차적으로 연결**하여 실행 할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 이러한 체인구조를 사용하면 **작업흐름이 명확**해지고 **코드의 재사용성**이 높아지며 **유지 보수 및 확장성이** 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 각 클래스는 다양한 chain 알고리즘들을 미리 구현한 것으로 상황에 맞는 것을 선택하여 필요한 구성요소를 전달해 생생한다.
- 이 방식은 LangChain의 **초기 방식**이며, 새로운 기능 확장이나 유연한 구성에 한계가 있기 때문에 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.
  - 현재 LangChain에서는 권장하지 않는 방식이다.

### 2. LCEL (LangChain Expression Language) 방식

- LCEL은 체인을 표현식(Expression) 기반의 선언적 파이프라인 방식으로 구성할 수 있도록 설계된 최신 체인 구성 방법이다. 
- 각 컴포넌트들을 `|` 연산자로 연결하여, 흐름이 자연스럽게 이어지는 형태의 체인을 구성한다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 융통성과 확장성 있는 체인 구성**이 가능하다.
- LCEL은
  - 선형적 흐름 구조를 가진다.
  - 문법이 간결하고 선언적이다.
  - 체인의 구조가 코드만 봐도 쉽게 파악된다.
  - 유연하고 확장성이 매우 뛰어나다.
- `Runnable` 기반 구조
  - LCEL방식을 구성하는 모든 컴포넌트들은 `Runnable` 이라는 공통 인터페이스를 기반으로 동작한다.
  - 체인을 구성하는 각 컴포넌트들은 `Runnable` 을 상속하여 구현하여 이를 통해 일관된 실행 인터페이스를 제공한다.
  - **공통 메소드**:
    - `invoke()`: 단일 입력에 대한 처리
    - `batch()`: 다수 입력을 묶어서 한번에 처리
    - `stream()`: 스트리밍 방식의 요청
    - `ainvoke()`, `abatch()`, `astream()`: 비동기적 처리 메소드

# Runnable 타입 주요 클래스

## [Runnable](https://reference.langchain.com/python/langchain_core/runnables/#langchain_core.runnables.base.Runnable)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - **`invoke(input, config:RunnableConfig)->output`**: 단일 입력을 처리하여 결과를 반환.
    - **`batch(input:list, config:RunnableConfig|list[RunnableConfig]) -> list[Output]`**: 여러 입력 데이터들을 한 번에 처리.
    - **`stream(input, config:RunnableConfig) -> Iterator[Output]`**: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - **`assign(**kwargs)`**:
      -  앞 Runnable의 출력 결과에 새로운 key–value 쌍의 Field 추가(assign) 하여 다음 Runnable로 전달.
      -  값으로는 Runnable 객체(LCEL체인등)나 고정 값(리터럴) 모두 가능하며, 각 항목은 실행 시 평가되어 기존 출력에 병합한다.
      -  주로 앞 단계의 출력에 부가 정보(field)를 추가하고자 할 때 사용한다. 특히 `RunnablePassthrough`와 결합해, 입력을 그대로 넘기면서 특정 field만 추가할 때 자주 사용
### Runnable의 주요 구현체(하위 클래스)

- **`RunnableSequence`**
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - 보통은 LCEL 문법을 사용해서 정의한다.
      - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
  
-  **`RunnablePassthrough`**
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
      - 앞 Runnable으로 부터 전달 받은 **입력 값을 다음 Runnable로 그대로 전달**한다.
           - `RunnablePassthrough()`
      - 입력받은 값에 **Field를 추가**해서 전달할 경우 `assign()` 메소드를 사용한다.
           - `RunnablePassthrough.assign(new_key1="new_value1", new_key2="new_value2", ..)`

- **`RunnableParallel`**
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 
        ```python
        RunnableParallel(
            {
                "key1":Runnable1, 
                "key2":Runnable2,
                "key3":Runnable3, ...
            }
        )
        ```
      - 각 Runnable의 실행결과를 Value로 Dictionary를 생성해서 반환한다.
      - LCEL로 정의할 때는 Chain에 dictionary로 정의한다.

- **`RunnableLambda`**
    - 일반 함수를 `Runnable`로 변환할 때 사용한다.
    
    - Runnable을 입력해야 하는 자리에 함수를 넣어야 하는 경우 RunnableLambda로 그 함수를 Runnable로 만들어 넣는다.
    - **구현**
      1. `RunnableLambda(함수객체)`
      2. `@chain` decorator를 이용해 함수를 `RunnableLamda`로 구현할 수있다.
      3. LCEL 에 함수를 포함시키면 `RunnableLambda`로 자동 변환된다.

#### Runnable 예제

#### RunnableLambda 예제

#### RunnablePassThrough 예제

#### RunnableParallel 예제

### LCEL Chain 예제

### Chain과 Chain간의 연결

## 사용자 함수를 Chain에 적용하기

### 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.
  - 보통 Lambda 표현식으로 정의한 함수를 LCEL chain에 포함시킬 때 사용한다.
  
#### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

### 함수 자체를 chain에 추가
- RunnableLambda에 전달할 함수 구문에 맞는 함수라면 RunnableLambda를 사용하지 않고 chain에 넣을 수 있다. 
- 단 함수로 정의하고 LCEL에 포함시켜야 한다. Lambda 표현식으로 작성한 함수는 `RunnableLambda`를 사용해야 한다.

### 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```